In [14]:
# base model, not tuned

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Embedding
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv("../data/processed/preprocessed_reviews.csv")

# tokenize
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

# replace out-of-vocab with 0
sequences = [[token if token < 3000 else 0 for token in seq] for seq in sequences]
X = pad_sequences(sequences, maxlen=300)
y = df['sentiment_binary'].values

# split into 80:20 train:test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

df.head()

vocab_size = 10000
embedding_dim = 128
lstm_out = 196
epochs = 5

# make model using adam optimizer
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

history = model.fit(X_train, y_train, 
                    epochs=epochs, verbose=1, 
                    validation_split=0.1, callbacks=[early_stopping])

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 125s 531ms/step - accuracy: 0.6277 - loss: 0.6629 - val_accuracy: 0.8279 - val_loss: 0.3791
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 127s 541ms/step - accuracy: 0.8695 - loss: 0.3292 - val_accuracy: 0.7557 - val_loss: 0.5018
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 125s 532ms/step - accuracy: 0.8722 - loss: 0.3203 - val_accuracy: 0.8496 - val_loss: 0.3622
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 126s 540ms/step - accuracy: 0.9083 - loss: 0.2367 - val_accuracy: 0.8436 - val_loss: 0.3768
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 123s 526ms/step - accuracy: 0.9273 - loss: 0.2080 - val_accuracy: 0.8496 - val_loss: 0.3998


In [15]:
import random
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
binary_predictions = (y_pred > 0.5).astype(int)

# convert y_test to a numpy array
y_test_array = np.array(y_test)

# get 10 random indices
random_indices = random.sample(range(len(y_test_array)), 10)

# loop through random indices
for idx in random_indices:
    review_text = df.loc[idx, 'cleaned_text']

    print(f"Review: {review_text[:200]}...")  # print first 200 chars of review
    print(f"True Label: {y_test_array[idx]}")  # print true sentiment of review (0 for negative, 1 for positive)
    print(f"Predicted Label: {binary_predictions[idx]}")  # print predicted sentiment of review (0 for negative, 1 for positive)
    print("------------------------------------------------------------------")

print("Accuracy:", accuracy_score(y_test, binary_predictions))

65/65 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step
Review: advisable reliable highly complicated...
True Label: 1.0
Predicted Label: [1]
------------------------------------------------------------------
Review: dont care first opened showed appointments thinking heck figured showing weeks passed along current thats bunch clutter telling five appointments month one showed one still says dont clue four anytime...
True Label: 0.0
Predicted Label: [0]
------------------------------------------------------------------
Review: weather kitten many different...
True Label: 1.0
Predicted Label: [1]
------------------------------------------------------------------
Review: constant prompts trials upgrades super annoying...
True Label: 0.0
Predicted Label: [0]
------------------------------------------------------------------
Review: happy app comes time durationother perfect app one easily create tasks...
True Label: 0.0
Predicted Label: [0]
----------------------------------------------------------------

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time

# load and shuffle data
unshuffled_df = pd.read_csv("../data/processed/preprocessed_reviews.csv")
unshuffled_df['original_index'] = unshuffled_df.index
df = unshuffled_df.sample(frac=1, random_state=42).reset_index(drop=True)
df['label'] = df['score'].apply(lambda x: 1 if x >= 4 else 0)

# tokenize
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])

# replace out-of-vocab with 0
sequences = [[token if token < 3000 else 0 for token in seq] for seq in sequences]
X = pad_sequences(sequences, maxlen=300)
y = df['label'].values

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# chosen hyperparameters
lr = 0.01
momentum = 0.9
lstm_units = 128
lstm_layers = 1
embedding_dim = 128
vocab_size = 10000

epochs = 100

print(f"\nTraining with lr={lr}, momentum={momentum}, lstm_units={lstm_units}, lstm_layers={lstm_layers}")
start_time = time.time()

# build model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
optimizer = SGD(learning_rate=lr, momentum=momentum)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# train model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=64,
                    validation_split=0.1, verbose=1)

# briefly evaluate model (more detailed evaluation in later cell)
val_acc = history.history['val_accuracy'][-1]
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

# calculate time took to train model
elapsed = time.time() - start_time
print(f"Validation Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}, Time: {elapsed:.2f} seconds")


Training with lr=0.01, momentum=0.9, lstm_units=128, lstm_layers=1
Epoch 1/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 41s 343ms/step - accuracy: 0.5068 - loss: 0.6916 - val_accuracy: 0.5319 - val_loss: 0.6845
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 40s 346ms/step - accuracy: 0.5529 - loss: 0.6784 - val_accuracy: 0.5909 - val_loss: 0.6610
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 40s 344ms/step - accuracy: 0.5968 - loss: 0.6660 - val_accuracy: 0.6005 - val_loss: 0.6594
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 41s 346ms/step - accuracy: 0.5948 - loss: 0.6624 - val_accuracy: 0.6065 - val_loss: 0.6561
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 41s 353ms/step - accuracy: 0.6072 - loss: 0.6577 - val_accuracy: 0.6101 - val_loss: 0.6510
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 41s 353ms/step - accuracy: 0.6226 - loss: 0.6506 - val_accuracy: 0.6005 - val_loss: 0.6520
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 42s 356ms/step - accuracy: 0.6291 - loss: 0.6476 - val_accuracy: 0.6282 - val_loss: 0.6291
Epoch 8/

In [21]:
import random
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
binary_predictions = (y_pred > 0.5).astype(int)

# convert y_test to a numpy array
y_test_array = np.array(y_test)

# get 10 random indices
random_indices = random.sample(range(len(y_test_array)), 10)

# loop through random indices
for idx in random_indices:
    review_text = unshuffled_df.loc[df.loc[idx, 'original_index'], 'content']

    print(f"Review: {review_text[:200]}...")  # print first 200 chars of review
    print(f"True Label: {y_test_array[idx]}")  # print true sentiment of review (0 for negative, 1 for positive)
    print(f"Predicted Label: {binary_predictions[idx]}")  # print predicted sentiment of review (0 for negative, 1 for positive)
    print("------------------------------------------------------------------")

print("Accuracy:", accuracy_score(y_test, binary_predictions))


65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step
Review: Nice 👍 one...
True Label: 0
Predicted Label: [0]
------------------------------------------------------------------
Review: This is great! After having played it for three days I was begging my mom for chores to do!...
True Label: 0
Predicted Label: [0]
------------------------------------------------------------------
Review: Difficult to use...
True Label: 1
Predicted Label: [0]
------------------------------------------------------------------
Review: Hum hai al Ghj...
True Label: 0
Predicted Label: [0]
------------------------------------------------------------------
Review: The backup feature does not work. I am stuck having to copy over 1000 items in my lists manually. I understand syncing is a premium feature, but the backup feature is not and I wish I hadn't bothered ...
True Label: 1
Predicted Label: [1]
------------------------------------------------------------------
Review: Excellent...
True Label: 0
Predicted Label: [0]
---

In [16]:
import os
import json
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# get predictions
pred_out = model.predict(X_test)
binary_predictions = (y_pred > 0.5).astype(int)
y_test_array = np.array(y_test)

# compute metrics
acc  = accuracy_score(y_test_array, binary_predictions)
prec = precision_score(y_test_array, binary_predictions)
rec  = recall_score(y_test_array, binary_predictions)
f1   = f1_score(y_test_array, binary_predictions)

clf_rep = classification_report(
    y_test_array, binary_predictions,
    target_names=["Negative", "Positive"],
    digits=4
)

# create evaluation directory if doesn't exist
os.makedirs("../evaluation", exist_ok=True)

# save numeric metrics as JSON
metrics = {
    "model": "lstm",
    "accuracy": acc,
    "precision": prec,
    "recall": rec,
    "f1": f1
}
with open("../evaluation/lstm_metrics_adam.json", "w") as fp:
    json.dump(metrics, fp, indent=2)

# create confusion matrix plo
cm = confusion_matrix(y_test_array, binary_predictions)
plt.figure(figsize=(5, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Neg","Pos"],
    yticklabels=["Neg","Pos"]
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Adam LSTM Confusion Matrix")
plt.tight_layout()
plt.savefig("../evaluation/lstm_adam_confusion_matrix.png")
plt.close()

print("evaluation metrics & plots saved to /evaluation/")

65/65 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step
evaluation metrics & plots saved to /evaluation/


In [37]:
# create models directory
os.makedirs("../models", exist_ok=True)

# save model
model.save('../models/lstm-sentiment.keras')
print("model saved to /models/")

model saved to /models/


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import time

# load and shuffle data
unshuffled_df = pd.read_csv("../data/processed/preprocessed_reviews.csv")
unshuffled_df['original_index'] = unshuffled_df.index
df = unshuffled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# tokenize
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

# replace out-of-vocab with 0
sequences = [[token if token < 3000 else 0 for token in seq] for seq in sequences]
X = pad_sequences(sequences, maxlen=300)
y = df['sentiment_binary'].values

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# chosen hyperparameters
lr = 0.01
momentum = 0.9
lstm_units = 128
lstm_layers = 1
embedding_dim = 128
vocab_size = 10000

epochs = 100

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

print(f"\nTraining with lr={lr}, momentum={momentum}, lstm_units={lstm_units}, lstm_layers={lstm_layers}")
start_time = time.time()

# build model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
optimizer = SGD(learning_rate=lr, momentum=momentum)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# train model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=64,
                    validation_split=0.1, verbose=1, callbacks=[early_stopping])

# briefly evaluate model (more detailed evaluation in later cell)
val_acc = history.history['val_accuracy'][-1]
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

# calculate time took to train model
elapsed = time.time() - start_time
print(f"Validation Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}, Time: {elapsed:.2f} seconds")


Training with lr=0.01, momentum=0.9, lstm_units=128, lstm_layers=1
Epoch 1/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 43s 368ms/step - accuracy: 0.5076 - loss: 0.6908 - val_accuracy: 0.5319 - val_loss: 0.6819
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 44s 374ms/step - accuracy: 0.5511 - loss: 0.6790 - val_accuracy: 0.5872 - val_loss: 0.6661
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 43s 370ms/step - accuracy: 0.6005 - loss: 0.6670 - val_accuracy: 0.5884 - val_loss: 0.6609
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 44s 372ms/step - accuracy: 0.5895 - loss: 0.6677 - val_accuracy: 0.5993 - val_loss: 0.6569
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 44s 375ms/step - accuracy: 0.5907 - loss: 0.6627 - val_accuracy: 0.6197 - val_loss: 0.6526
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 43s 371ms/step - accuracy: 0.6123 - loss: 0.6554 - val_accuracy: 0.6342 - val_loss: 0.6461
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 44s 380ms/step - accuracy: 0.6272 - loss: 0.6440 - val_accuracy: 0.6486 - val_loss: 0.6312
Epoch 8/

In [16]:
import random
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
binary_predictions = (y_pred > 0.5).astype(int)

# convert y_test to a numpy array
y_test_array = np.array(y_test)

# get 10 random indices
random_indices = random.sample(range(len(y_test_array)), 10)

# loop through random indices
for idx in random_indices:
    review_text = unshuffled_df.loc[idx, 'cleaned_text']

    print(f"Review: {review_text[:200]}...")  # print first 200 chars of review
    print(f"True Label: {y_test_array[idx]}")  # print true sentiment of review (0 for negative, 1 for positive)
    print(f"Predicted Label: {binary_predictions[idx]}")  # print predicted sentiment of review (0 for negative, 1 for positive)
    print("------------------------------------------------------------------")

print("Accuracy:", accuracy_score(y_test, binary_predictions))


65/65 ━━━━━━━━━━━━━━━━━━━━ 18s 281ms/step
Review: really help manage day...
True Label: 1.0
Predicted Label: [1]
------------------------------------------------------------------
Review: annoys user rate app heres one star...
True Label: 0.0
Predicted Label: [0]
------------------------------------------------------------------
Review: need subscrib app use make alot thing lock unlock need subscrib...
True Label: 1.0
Predicted Label: [0]
------------------------------------------------------------------
Review: app good differences premium free version arent mentioned beginning also possible start using app without free trial period far seen forced free trial period realize sync function enables use app simu...
True Label: 1.0
Predicted Label: [1]
------------------------------------------------------------------
Review: nice app ben using years...
True Label: 0.0
Predicted Label: [0]
------------------------------------------------------------------
Review: syncs everything terrible 

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import time

# load data
df = pd.read_csv("../data/processed/preprocessed_reviews.csv")

# tokenize
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

# replace out-of-vocab with 0
sequences = [[token if token < 3000 else 0 for token in seq] for seq in sequences]
X = pad_sequences(sequences, maxlen=300)
y = df['sentiment_binary'].values

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# chosen hyperparameters
lr = 0.01
momentum = 0.9
lstm_units = 256
lstm_layers = 1
embedding_dim = 128
vocab_size = 10000

epochs = 100

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

print(f"\nTraining with lr={lr}, momentum={momentum}, lstm_units={lstm_units}, lstm_layers={lstm_layers}")
start_time = time.time()

# build model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
optimizer = SGD(learning_rate=lr, momentum=momentum)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# train model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=64,
                    validation_split=0.1, verbose=1, callbacks=[early_stopping])

# briefly evaluate model (more detailed evaluation in later cell)
val_acc = history.history['val_accuracy'][-1]
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

# calculate time took to train model
elapsed = time.time() - start_time
print(f"Validation Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}, Time: {elapsed:.2f} seconds")


Training with lr=0.01, momentum=0.9, lstm_units=256, lstm_layers=1
Epoch 1/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 84s 714ms/step - accuracy: 0.5218 - loss: 0.6900 - val_accuracy: 0.5319 - val_loss: 0.6824
Epoch 2/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 87s 748ms/step - accuracy: 0.5446 - loss: 0.6794 - val_accuracy: 0.5848 - val_loss: 0.6659
Epoch 3/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 739ms/step - accuracy: 0.5838 - loss: 0.6693 - val_accuracy: 0.6077 - val_loss: 0.6617
Epoch 4/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 83s 707ms/step - accuracy: 0.6028 - loss: 0.6630 - val_accuracy: 0.6101 - val_loss: 0.6587
Epoch 5/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 77s 657ms/step - accuracy: 0.5976 - loss: 0.6658 - val_accuracy: 0.5969 - val_loss: 0.6580
Epoch 6/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 78s 666ms/step - accuracy: 0.6147 - loss: 0.6485 - val_accuracy: 0.6318 - val_loss: 0.6470
Epoch 7/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 78s 667ms/step - accuracy: 0.6263 - loss: 0.6475 - val_accuracy: 0.6378 - val_loss: 0.6355
Epoch 8/

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import time

# load data
df = pd.read_csv("../data/processed/preprocessed_reviews.csv")

# tokenize
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

# replace out-of-vocab with 0
sequences = [[token if token < 3000 else 0 for token in seq] for seq in sequences]
X = pad_sequences(sequences, maxlen=300)
y = df['sentiment_binary'].values

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# chosen hyperparameters
lr = 0.01
momentum = 0.9
lstm_units = 256
lstm_layers = 1
embedding_dim = 128
vocab_size = 10000

epochs = 100

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

print(f"\nTraining with lr={lr}, momentum={momentum}, lstm_units={lstm_units}, lstm_layers={lstm_layers}")
start_time = time.time()

# build model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(Bidirectional(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
optimizer = SGD(learning_rate=lr, momentum=momentum)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# train model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=64,
                    validation_split=0.1, verbose=1, callbacks=[early_stopping])

# briefly evaluate model (more detailed evaluation in later cell)
val_acc = history.history['val_accuracy'][-1]
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

# calculate time took to train model
elapsed = time.time() - start_time
print(f"Validation Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}, Time: {elapsed:.2f} seconds")


Training with lr=0.01, momentum=0.9, lstm_units=256, lstm_layers=1
Epoch 1/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 117s 993ms/step - accuracy: 0.5156 - loss: 0.6912 - val_accuracy: 0.5211 - val_loss: 0.6897
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.5412 - loss: 0.6856 - val_accuracy: 0.5235 - val_loss: 0.6839
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.5592 - loss: 0.6779 - val_accuracy: 0.5608 - val_loss: 0.6772
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.5804 - loss: 0.6737 - val_accuracy: 0.5632 - val_loss: 0.6760
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 173s 1s/step - accuracy: 0.5947 - loss: 0.6677 - val_accuracy: 0.5909 - val_loss: 0.6724
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - accuracy: 0.5917 - loss: 0.6639 - val_accuracy: 0.5969 - val_loss: 0.6696
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - accuracy: 0.6115 - loss: 0.6566 - val_accuracy: 0.6041 - val_loss: 0.6641
Epoch 8/100
117/117

In [12]:
import random
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
binary_predictions = (y_pred > 0.5).astype(int)

# convert y_test to a numpy array
y_test_array = np.array(y_test)

# get 10 random indices
random_indices = random.sample(range(len(y_test_array)), 10)

# loop through random indices
for idx in random_indices:
    review_text = df.loc[idx, 'cleaned_text']

    print(f"Review: {review_text[:200]}...")  # print first 200 chars of review
    print(f"True Label: {y_test_array[idx]}")  # print true sentiment of review (0 for negative, 1 for positive)
    print(f"Predicted Label: {binary_predictions[idx]}")  # print predicted sentiment of review (0 for negative, 1 for positive)
    print("------------------------------------------------------------------")

print("Accuracy:", accuracy_score(y_test, binary_predictions))


65/65 ━━━━━━━━━━━━━━━━━━━━ 19s 288ms/step
Review: good list person...
True Label: 1.0
Predicted Label: [1]
------------------------------------------------------------------
Review: free forever reminders premium feature hmm dyspraxic without much money waste looking todo list app help without reminders app functionally useless disappointed youve opted contentcarve extent argue c...
True Label: 0.0
Predicted Label: [0]
------------------------------------------------------------------
Review: great tool use every day theres bug however thats annoying galaxy note move items theres chance item loose previously assigned datetime moved someday list annoying time consuming sorting mode time add...
True Label: 1.0
Predicted Label: [1]
------------------------------------------------------------------
Review: sends mails claiming monthly payments without yearly commitment monthly without yearly commitment costs also app quite crappy without premium premium would still expensive get...
True La

In [14]:
import os
import json
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# get predictions
pred_out = model.predict(X_test)
binary_predictions = (y_pred > 0.5).astype(int)
y_test_array = np.array(y_test)

# compute metrics
acc  = accuracy_score(y_test_array, binary_predictions)
prec = precision_score(y_test_array, binary_predictions)
rec  = recall_score(y_test_array, binary_predictions)
f1   = f1_score(y_test_array, binary_predictions)

clf_rep = classification_report(
    y_test_array, binary_predictions,
    target_names=["Negative", "Positive"],
    digits=4
)

# create evaluation directory if doesn't exist
os.makedirs("../evaluation", exist_ok=True)

# save numeric metrics as JSON
metrics = {
    "model": "lstm",
    "accuracy": acc,
    "precision": prec,
    "recall": rec,
    "f1": f1
}
with open("../evaluation/lstm_bidirectional_metrics.json", "w") as fp:
    json.dump(metrics, fp, indent=2)

# create confusion matrix plo
cm = confusion_matrix(y_test_array, binary_predictions)
plt.figure(figsize=(5, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Neg","Pos"],
    yticklabels=["Neg","Pos"]
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Bidirectional LSTM Confusion Matrix")
plt.tight_layout()
plt.savefig("../evaluation/lstm_bidirectional_confusion_matrix.png")
plt.close()

print("evaluation metrics & plots saved to /evaluation/")

65/65 ━━━━━━━━━━━━━━━━━━━━ 18s 270ms/step
evaluation metrics & plots saved to /evaluation/


In [15]:
# create models directory
os.makedirs("../models", exist_ok=True)

# save model
model.save('../models/lstm-sentiment-bidirectional.keras')
print("model saved to /models/")

model saved to /models/
